## 此文件请在Google Colab中运行

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/')

Mounted at /content/drive


In [ ]:
!pip install -U LibRecommender

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from libreco.data import random_split, DatasetPure
from libreco.algorithms import LightGCN  # pure data, algorithm LightGCN
from libreco.algorithms import NGCF
from libreco.algorithms import GraphSage
from libreco.algorithms import SVD
from libreco.evaluation import evaluate

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/打分数据.csv")
#data = data.rename(columns={'userId': 'user','movieId':'item','rating': 'label','timestap':'time'})
#data = data.rename(columns={'userId': 'user','movieId':'item','rating': 'label'})
data = data.rename(columns={'User ID': 'user', 'Food ID':'item','Rating':'label'})

# split whole data into three folds for training, evaluating and testing
train_data, eval_data, test_data = random_split(data, multi_ratios=[0.8, 0.1, 0.1])

train_data, data_info = DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)
print(data_info)  # n_users: 5894, n_items: 3253, data sparsity: 0.4172 %

svd = NGCF(
    task="ranking",
    data_info=data_info,
    loss_type="bpr",
    embed_size=16,
    n_epochs=3,
    lr=1e-3,
    batch_size=2048,
    num_neg=1,
    device="cuda",
)
# monitor metrics on eval data during training
svd.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    eval_data=eval_data,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)

# do final evaluation on test data
evaluate(
    model=svd,
    data=test_data,
    neg_sampling=True,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)



n_users: 142, n_items: 50, data density: 8.0000 %
Training start time: 2024-04-22 18:24:17


train: 100%|██████████| 1/1 [00:00<00:00,  5.88it/s]


Epoch 1 elapsed: 0.179s
	 train_loss: 0.5541


eval_listwise: 100%|██████████| 1/1 [00:00<00:00, 61.22it/s]


	 eval log_loss: 0.9249
	 eval roc_auc: 0.5552
	 eval precision@10: 0.0263
	 eval recall@10: 0.2281
	 eval ndcg@10: 0.1279


...

Epoch 3 elapsed: 0.042s
	 train_loss: 0.4784


eval_listwise: 100%|██████████| 1/1 [00:00<00:00, 150.18it/s]


	 eval log_loss: 0.8462
	 eval roc_auc: 0.5469
	 eval precision@10: 0.0228
	 eval recall@10: 0.2018
	 eval ndcg@10: 0.1166


eval_listwise: 100%|██████████| 1/1 [00:00<00:00, 121.75it/s]


{'loss': 0.8354567171892381,
 'roc_auc': 0.573100575282682,
 'precision': 0.03333333333333333,
 'recall': 0.29532163742690054,
 'ndcg': 0.13584542562106872}

In [ ]:
# recommend 5 items for user 1
svd.recommend_user(user=1, n_rec=5)

{1: array(['黄焖茄子', '烧鸭手撕鸡饭', '番茄无骨鱼', '荠菜鲜肉小馄饨', '脆皮锅烧粉'], dtype='<U7')}

In [ ]:
# recommend 5 items for user 2
svd.recommend_user(user=2, n_rec=5)

{2: array(['肉多多汤粉', '荠菜鲜肉小馄饨', '蜜汁鸡腿饭', '酸汤肥牛', '黄焖鸡'], dtype='<U7')}

In [ ]:
# recommend 5 items for user 3
svd.recommend_user(user=3, n_rec=5)

{3: array(['蜜汁鸡腿饭', '土耳其烤肉拌饭', '重庆酸辣粉', '番茄无骨鱼', '酸汤肥牛'], dtype='<U7')}